### Evaluate variables.

In [ ]:
%clear -f
import keras.backend as K

x = K.variable(42.)

# Solution 1:
sess = K.get_session()
print sess.run(x)

# Solution 2:
print K.eval(x)

### Feed placeholders and evaluate functions.

In [ ]:
%clear -f
import keras.backend as K

x = K.placeholder(ndim=1)
y = 2 * x
feed = np.array([2])

# Solution 1:
sess = K.get_session()
print sess.run(y, { x : feed})

# Solution 2 (general for tensorflow and theano):
f = K.function([x], [y])
print f([feed])[0]

### Compute custom gradients.

In [27]:
%clear -f
import numpy as np
import keras.backend as K
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras import optimizers, losses

# Define the model and generate the data.
print "Our model is y = 3 * x1 + 2 * x2"
num_data = 1000
x_train = np.random.rand(num_data, 2)
y_train = x_train * np.matrix([[3], [2]])
print "Data generated."
x = Input(shape=(2,), name='x')
y = Dense(1, activation='linear', use_bias=False)(x)
model = Model(inputs=x, outputs=y)

opt = optimizers.Adam(0.1)
loss = losses.mean_squared_error

model.compile(opt, loss)

# Retreive the handle to trainable weights, which should be equal to the dy_dx.
w = model.trainable_weights[0]

dy_dx = K.gradients(y, x) 
f = K.function(inputs=[x], outputs=dy_dx)
fx = f([np.ones([1, 2])])[0]  # Input is irrelevant, since we are computing the gradient of a linear function.
print "Before training:"
print "dy_dx = ", fx, ", w = ", K.eval(w).T 
model.fit(x_train, y_train, epochs=500, batch_size=num_data, verbose=0)
print "\nAfter training:"
print "dy_dx = ", f([np.ones([1, 2])])[0], ", w = ", K.eval(w).T 

# Calculate the gradient manually.
y_true = K.placeholder(shape=(None, 1))
dJ_dw = K.gradients(loss(y, y_true), w) 
f = K.function(inputs=[x, y_true], outputs=dJ_dw)

print "\ndJ_dw (should be around 0):"
print f([x_train, y_train])[0].T

Our model is y = 3 * x1 + 2 * x2
Data generated.
Before training:
dy_dx =  [[-0.60748106  0.92611754]] , w =  [[-0.60748106  0.92611754]]

After training:
dy_dx =  [[ 3.00000048  1.99999976]] , w =  [[ 3.00000048  1.99999976]]

dJ_dw (should be around 0):
[[  1.88174570e-04   8.23366136e-05]]


### Pass a custom gradient to the optimizer.

In [51]:
%clear -f
import tensorflow as tf
import numpy as np
import keras.backend as K
from keras.layers import Input, Dense, Activation
from keras.models import Model
from keras import optimizers, losses

# Define the model and generate the data.
print "Our model is y = 3 * x1 + 2 * x2 + 5"
num_data = 1000
x_train = np.random.rand(num_data, 2)
y_train = x_train * np.matrix([[3], [2]]) + 5
print "Data generated."
x = Input(shape=(2,), name='x')
y = Dense(1, activation='linear')(x)
model = Model(inputs=x, outputs=y)

# Retreive the handle to trainable weights. In this case it is a list of variables [W, b].
w = model.trainable_weights
store_w = {l: l.get_weights() for l in model.layers}

print "Initial weights:", [K.eval(i).T for i in w]

# opt = optimizers.Adam(0.1)
opt = tf.train.AdamOptimizer(0.1)
# Output placeholder is needed for to define the loss.
y_true = K.placeholder(shape=(None, 1))
loss = losses.mean_squared_error(y, y_true)

grads_and_vars = zip(tf.gradients(loss, w), w)
op = opt.apply_gradients(grads_and_vars)

print "\nFirst approach:"
f = K.function(inputs=[x, y_true], outputs=[op])
for i in range(1000):
    f([x_train, y_train])
    
print "Optimized weights:", [K.eval(i).T for i in w]

# Restore weights from beginning:
for l in model.layers:
    l.set_weights(store_w[l])
print "\nRestored weights:", [K.eval(i).T for i in w]

print "\nSecond approach:"
sess = K.get_session()
sess.run(tf.global_variables_initializer())
for i in range(1000):
    sess.run(op, {x : x_train, y_true : y_train})
    
print "Optimized weights:", [K.eval(i).T for i in w]

Our model is y = 3 * x1 + 2 * x2 + 5
Data generated.
Initial weights: [array([[-0.52793747, -1.14754939]], dtype=float32), array([ 0.], dtype=float32)]

First approach:
Optimized weights: [array([[ 3.00197625,  2.00161314]], dtype=float32), array([ 4.99804544], dtype=float32)]

Restored weights: [array([[-0.52793747, -1.14754939]], dtype=float32), array([ 0.], dtype=float32)]

Second approach:
Optimized weights: [array([[ 3.00096726,  2.00076246]], dtype=float32), array([ 4.99905729], dtype=float32)]
